In [1]:
!wget https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip

--2021-02-14 07:16:08--  https://www.dropbox.com/s/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.83.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.83.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip [following]
--2021-02-14 07:16:09--  https://www.dropbox.com/s/raw/pdhwlpi2yeie0ol/movie-reviews-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2b4c82fbc33bfdb4f8ab615660.dl.dropboxusercontent.com/cd/0/inline/BI7qMuJgCJqRWr3sK6Y03FJfwl1vVj-GO-maym42pJaJx0fm6vQ9kZxWZgzEEEhPEZU-gCjivKKL1AxWFp2xVhbDhXtogMFTfqtbq700doJkD43VuOPEsEwybhttDbu2tA0/file# [following]
--2021-02-14 07:16:10--  https://uc2b4c82fbc33bfdb4f8ab615660.dl.dropboxusercontent.com/cd/0/inline/BI7qMuJgCJqRWr3sK6Y03FJfwl1vVj-GO-maym42pJaJx0fm6vQ9kZxWZgzEEEhPEZU-gC

In [2]:
!unzip -q "/content/movie-reviews-dataset.zip"

In [3]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
from tensorflow.strings import regex_replace
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, GRU, Embedding, Dropout , Bidirectional

In [4]:
def prepareData(dir):
  data = text_dataset_from_directory(dir)
  return data.map(
    lambda text, label: (regex_replace(text, '<br />', ' '), label),
  )

In [5]:
train_data = prepareData('movie-reviews-dataset/train')
test_data = prepareData('movie-reviews-dataset/test')

for text_batch, label_batch in train_data.take(1):
  print(text_batch.numpy()[0])
  print(label_batch.numpy()[0])

Found 25000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.
b'From the critical acclaim, I expected more from this movie and from Tamara Jenkins. The story just meandered along and didn\'t seem to have a point or a plot. And I find it hard to believe that a 14 year old girl (mature for her age or no) would be so blase about getting the loss of her virginity "over with." Maybe I am too young to relate (I was four years old in 1976), but I didn\'t have any problems connecting with the stories of Shakespeare in Love or Life is Beautiful and I wasn\'t alive for either of those settings. The cast is very good but unfortunately for them the script did not alow them to engage the audience. Overall, Slums had its moments but unless you are yearning to reminisce over halter tops and tube socks, I would say skip this one.'
0


In [6]:
model = Sequential()

In [7]:
model.add(Input(shape=(1,), dtype="string"))

In [8]:
max_tokens = 1000
max_len = 100
vectorize_layer = TextVectorization(
  max_tokens=max_tokens,
  output_mode="int",
  output_sequence_length=max_len,
)

In [9]:
train_texts = train_data.map(lambda text, label: text)
vectorize_layer.adapt(train_texts)

model.add(vectorize_layer)

In [10]:
model.add(Embedding(max_tokens + 1, 128))

model.add(Bidirectional(GRU(64)))
model.add(Dense(64, activation="relu"))
model.add(Dense(1, activation="sigmoid"))

In [11]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
model.fit(train_data, epochs=10)

Epoch 1/10
782/782 [==============================] - 30s 27ms/step - loss: 0.5936 - accuracy: 0.6545
Epoch 2/10
782/782 [==============================] - 22s 28ms/step - loss: 0.4337 - accuracy: 0.7987
Epoch 3/10
782/782 [==============================] - 22s 28ms/step - loss: 0.3926 - accuracy: 0.8228
Epoch 4/10
782/782 [==============================] - 22s 28ms/step - loss: 0.3594 - accuracy: 0.8360
Epoch 5/10
782/782 [==============================] - 21s 27ms/step - loss: 0.3337 - accuracy: 0.8529
Epoch 6/10
782/782 [==============================] - 21s 27ms/step - loss: 0.3086 - accuracy: 0.8669
Epoch 7/10
782/782 [==============================] - 22s 28ms/step - loss: 0.2818 - accuracy: 0.8768
Epoch 8/10
782/782 [==============================] - 22s 28ms/step - loss: 0.2569 - accuracy: 0.8891
Epoch 9/10
782/782 [==============================] - 22s 29ms/step - loss: 0.2304 - accuracy: 0.9006
Epoch 10/10
782/782 [==============================] - 22s 28ms/step - loss: 0.202

In [13]:
model.evaluate(test_data)

782/782 [==============================] - 15s 18ms/step - loss: 0.8693 - accuracy: 0.7642


[0.8692539930343628, 0.764240026473999]

In [14]:
text = "I loved the movie !"

In [15]:
model.predict([text])

array([[0.9597331]], dtype=float32)